In [15]:
from datetime import datetime
import pandas as pd
import json
import requests
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 12]

In [16]:
import networkx as nx

from networkx.readwrite import json_graph
from networkx.algorithms import community

In [17]:
df = pd.read_csv('2008.csv')[['Month', 'DayofMonth', 'Year', 'ArrTime', 'Origin', 'Dest', 'Cancelled', 'FlightNum']]

In [18]:
df['Date'] = df['Year'].astype(str) +"-"+ df['Month'].astype(str) +"-"+ df['DayofMonth'].astype(str)

In [19]:
df.head()

,Month,DayofMonth,Year,ArrTime,Origin,Dest,Cancelled,FlightNum,Date
0,1,3,2008,2211.0,IAD,TPA,0,335,2008-1-3
1,1,3,2008,1002.0,IAD,TPA,0,3231,2008-1-3
2,1,3,2008,804.0,IND,BWI,0,448,2008-1-3
3,1,3,2008,1054.0,IND,BWI,0,1746,2008-1-3
4,1,3,2008,1959.0,IND,BWI,0,3920,2008-1-3


In [20]:
df = df.loc[df['Cancelled'] != 1]
df = df.drop(['Month', 'DayofMonth', 'Year', 'Cancelled'], axis=1)

In [21]:
df.head()

,ArrTime,Origin,Dest,FlightNum,Date
0,2211.0,IAD,TPA,335,2008-1-3
1,1002.0,IAD,TPA,3231,2008-1-3
2,804.0,IND,BWI,448,2008-1-3
3,1054.0,IND,BWI,1746,2008-1-3
4,1959.0,IND,BWI,3920,2008-1-3


In [22]:
df = df.dropna()
df['ArrTime'] = df['ArrTime'].astype(int)
df['ArrTime'] = df['ArrTime'].astype(str)

In [23]:
df['ArrTime'] = ('0'*3) + df['ArrTime']
df['ArrTime'] = df['ArrTime'].str.replace('.*(?=....$)', '', regex=True) # оставим только последние 4 символа 
df['ArrTime'] = df['ArrTime'].str.replace('(?<=^..)(?!.$)', ':', regex=True)
df['ArrTime'] = df['ArrTime'].str.replace('^24', '00', regex=True)
df['ArrTime'] = df['ArrTime'] + ":00"
df['Date'] = df['Date'] + " " + df['ArrTime']

In [25]:
df['Date'] = df['Date'].astype('datetime64[ns]')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6858079 entries, 0 to 7009727
Data columns (total 5 columns):
 #   Column     Dtype         
---  ------     -----         
 0   ArrTime    object        
 1   Origin     object        
 2   Dest       object        
 3   FlightNum  int64         
 4   Date       datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 313.9+ MB


In [27]:
df.head()

,ArrTime,Origin,Dest,FlightNum,Date
0,22:11:00,IAD,TPA,335,2008-01-03 22:11:00
1,10:02:00,IAD,TPA,3231,2008-01-03 10:02:00
2,08:04:00,IND,BWI,448,2008-01-03 08:04:00
3,10:54:00,IND,BWI,1746,2008-01-03 10:54:00
4,19:59:00,IND,BWI,3920,2008-01-03 19:59:00


In [28]:
df = df.sort_values(by=['Date']) 

In [92]:
import random
from tqdm import tqdm

# установим порог, после которого симуляция останавливается
def start_infection(airport, prob, df, threshold=90):
    airports = pd.Series(df['Dest'].unique())
    spread_infection = pd.DataFrame(airports, columns=['airport'])
    spread_infection['infection'] = 0
    spread_infection.loc[spread_infection['airport'] == airport, 'infection'] = 1
    
    for i in tqdm(df.itertuples()):
        
        airport_of_origin = i[2]
        airport_of_dest = i[3]
        infected_or_not_orig = bool(spread_infection.loc[spread_infection['airport'] == airport_of_origin, 'infection'].all())
        infected_or_not_dest = bool(spread_infection.loc[spread_infection['airport'] == airport_of_dest, 'infection'].all())
        
        spread_infection.loc[spread_infection['airport'] == airport, 'infection'] = 1
        
        # нас не интересуют рейсы в заразные города
        if infected_or_not_dest:
            continue 
        
        if infected_or_not_orig and not infected_or_not_dest:
            if random.random() < prob:
                spread_infection.loc[spread_infection['airport'] == airport_of_dest, 'infection'] = 1
                spread_infection.loc[spread_infection['airport'] == airport_of_dest, 'infection_date'] = i[5]
        
        prop_infected = (spread_infection['infection'].value_counts(normalize=True) * 100)[1]
        
        if (spread_infection['infection'] == 1).all():
            print('Evil is WON!!!')
            return spread_infection
        
        # Проведя итераци со стопроцентым шансом заражения, ВСЕ аэропорты "заражались" только под конец датасета,
        # поэтому остановим симуляцию когда заразится 90 процентов от всех аэропортов.
        if np.round(prop_infected, 2) >= threshold: 
            print(f'{np.round(prop_infected)}% is infected!')
            return spread_infection
        

In [93]:
results = start_infection('FAR', 1, df)
results.dropna()

15494it [00:14, 1054.89it/s]

90.0% is infected!


,airport,infection,infection_date
0,CLT,1,2008-01-01 08:03:00
1,JAX,1,2008-01-01 09:08:00
2,HRL,1,2008-01-01 11:39:00
3,STL,1,2008-01-01 08:05:00
4,BOI,1,2008-01-01 09:30:00
...,...,...,...
269,LCH,1,2008-01-01 17:44:00
270,AVP,1,2008-01-01 18:18:00
271,MEI,1,2008-01-01 19:34:00
272,MCN,1,2008-01-01 20:03:00
